Many assays contained a small amount of proteins, insufficient to represent a cell line proteome. These are originated from fractionated samples loaded separately on the LC-MS/MS and thus giving unique raw-files for each fraction. These are pooled here after manual annotation of these fractions (shown in *Format_pool.csv*), so every assay represents a full proteome representation of a cell line.

In [1]:
import pandas as pd
import mysql.connector
import numpy as np

In [2]:
conn = mysql.connector.connect(user='root', password='password', host='127.0.0.1', port='3306',database='expression_atlas_cells')
mycursor = conn.cursor(buffered = True)

# check the connection
if conn.is_connected():
    print("connection succesfull")
else:
    print("no connection")

connection succesfull


In [105]:
#This shows an approximate of Trembl proteins in the database. These wont be used in the NSAF atlas.
tine_file_path = "/home/compomics/Sam/git/python/Reference/Database/"
uniprot = pd.read_csv(tine_file_path + "Uniprot.csv", sep = ",")

query = "SELECT DISTINCT(uniprot_id) FROM protein"
proteins_db = pd.read_sql(query, conn)

print(uniprot.shape, proteins_db.shape)

print(uniprot[~uniprot.uniprot_id.isin(proteins_db.uniprot_id)].count())
print(proteins_db[~proteins_db.uniprot_id.isin(uniprot.uniprot_id)].count())

(20385, 3) (22746, 1)
uniprot_id     3492
description    3492
length         3492
dtype: int64
uniprot_id    5853
dtype: int64


In [106]:
assaysql = "SELECT assay_id, peptide_id, quantification FROM peptide_to_assay"
assayData = pd.read_sql_query(assaysql, conn)
assayData.head()

,assay_id,peptide_id,quantification
0,30960,110730450,2.0
1,30961,110730450,2.0
2,30995,110730450,1.0
3,31006,110730450,1.0
4,31007,110730450,1.0


Get all peptide to protein relations

In [107]:
pepsql = "SELECT peptide_to_protein.peptide_id, peptide_to_protein.uniprot_id FROM peptide_to_protein"
pepData = pd.read_sql_query(pepsql, conn)
pepData.head()

,peptide_id,uniprot_id
0,122045961,A0A024QZ42
1,120203377,A0A024QZX5
2,122405701,A0A024R1R8
3,118966811,A0A024RA28
4,117787530,A0A024RAC6


Get sequence length for all proteins in the database

In [108]:
seqsql = "SELECT uniprot_id, length FROM protein WHERE length IS NOT NULL"
seqData = pd.read_sql_query(seqsql, conn)
seqData["length"] = pd.to_numeric(seqData['length'], errors = "coerce")
seqData.head()

,uniprot_id,length
0,A0A024RBG1,181.0
1,A0A075B6H7,116.0
2,A0A075B6I1,120.0
3,A0A075B6I3,123.0
4,A0A075B6L6,115.0


Select proteotypic peptides (peptide with 1 peptide to protein relation)

In [109]:
#Select proteotypic peptides
proteotypicData = pepData.groupby("peptide_id").filter(lambda x: len(x) == 1)
proteotypicData.head()

,peptide_id,uniprot_id
0,122045961,A0A024QZ42
1,120203377,A0A024QZX5
2,122405701,A0A024R1R8
3,118966811,A0A024RA28
4,117787530,A0A024RAC6


Select proteins which have more than 2 proteotypic peptides

In [110]:
#only select proteins with at least 2 proteotypic peptides
proteins = proteotypicData.groupby("uniprot_id").filter(lambda x: len(x) > 2)
proteins.shape

(411274, 2)

drop non human proteins

In [8]:
non_human_proteins = non_human = """ADH1_YEAST,,,
ALBU_BOVIN,,,
ALDOA_RABIT,,,
BGAL_ECOLI,,,
CAH2_BOVIN,,,
CAS1_BOVIN,,,
CAS2_BOVIN,,,
CASB_BOVIN,,,
CASK_BOVIN,,,
CYC_HORSE,,,
DHE3_BOVIN,,,
GAG_SCVLA,,,
GFP_AEQVI,,,
K1C15_SHEEP,,,
K1M1_SHEEP,,,
K1M2_SHEEP,,,
K2M1_SHEEP,,,
K2M2_SHEEP,,,
K2M3_SHEEP,,,
KRA3_SHEEP,,,
KRA61_SHEEP,,,
LALBA_BOVIN,,,
LYSC_CHICK,,,
LYSC_LYSEN,,,
MYG_HORSE,,,
REF_HEVBR,,,
SRPP_HEVBR,,,
TRY1_BOVIN,,,
TRYP_PIG""".split(",,,\n")
proteins = proteins[~proteins["uniprot_id"].isin(non_human_proteins)]

In [111]:
proteins = proteins[~proteins["uniprot_id"].str.contains(pat = r"[-_|]")]
proteins.shape

(410695, 2)

Merge assays containing spectral counts and proteins

In [112]:
#Merge assay table with spectral counts and table with proteins
protData = pd.merge(assayData, proteins, on = "peptide_id").sort_values(["assay_id", "uniprot_id"])
del protData['peptide_id']
protData.head(10)

,assay_id,quantification,uniprot_id
45210,30960,5.0,A0AV96
68526,30960,2.0,A0AVT1
87424,30960,2.0,A0AVT1
112763,30960,3.0,A0AVT1
44847,30960,3.0,A0FGR8
48683,30960,2.0,A0FGR8
78299,30960,1.0,A0MZ66
47951,30960,7.0,A1L0T0
47759,30960,1.0,A1L170
26026,30960,1.0,A2A4G1


Now add the manually curated excel file with following characteristics:<br>
- contains assay_id, PXD_accession and new label, "pool_id".
    - 0 = no pooling needed
    - no_annotation = drop it
    - x = unique integer within a project which represents the pool.

In [11]:
#Create excel file
query = "SELECT DISTINCT(assay_id) FROM peptide_to_assay"
df1 = pd.read_sql(query, conn)
query = "SELECT assay_id, project_id, filename FROM assay"
df2 = pd.read_sql(query, conn)
query = "SELECT project_id, PXD_accession FROM project"
df3 = pd.read_sql(query, conn)
pool_df = pd.merge(df2[df2.assay_id.isin(df1.assay_id)], df3 ,how = "inner", on = "project_id")
pool_df.to_csv("format_pool2.csv", index = False)
print(pool_df.shape)
pool_df.head()

(1943, 4)


,assay_id,project_id,filename,PXD_accession
0,30960,1815,3B10-1,PXD000533
1,30961,1815,3B10-2,PXD000533
2,30962,1815,3B1-1,PXD000533
3,30963,1815,3B11-1,PXD000533
4,30964,1815,3B11-2,PXD000533


In [7]:
fp2 = pd.read_csv("format_pool2.csv")
fp2["pool_id"] = np.nan
pf1 = pd.read_csv("Format_pool.csv", sep = ";")
pf2 = pd.read_csv("/home/compomics/Sam/git/python/master_thesis/Database/parsed_manual_meta2.csv")
print(pf2.shape)
pf2 = pf2[~(pf2.pool_id == "unk")]
print(fp2.shape, pf1.shape, pf2.shape)

(3152, 12)
(1943, 5) (859, 8) (2157, 12)


In [8]:
import logging
logging.basicConfig(filename='pooling.log', level=logging.DEBUG)
def parse(filename):
    if filename in pf1.filename.tolist() and filename in pf2.RAW.tolist():
        print(filename,"present in both files.")
        return np.nan
    elif filename in pf1.filename.tolist():
        logging.info(f"{filename}: {pf1[pf1.filename == filename].pool_id.values[0]}")
        return pf1[pf1.filename == filename].pool_id.values[0]
    elif filename in pf2.RAW.tolist():
        logging.info(f"{filename}: {pf2[pf2.RAW == filename].pool_id.values[0]}")
        return pf2[pf2.RAW == filename].pool_id.values[0]
    
    print(filename, "not in file")
    return np.nan

In [9]:
fp2["pool_id"] = fp2.apply(lambda x: parse(x.filename), axis = 1)

In [116]:
fp2.columns

Index(['assay_id', 'project_id', 'filename', 'PXD_accession', 'pool_id'], dtype='object')

In [117]:
#pool_formatting = pd.read_csv("Format_pool.csv", sep = ";")
pool_formatting = fp2
pool_formatting = pool_formatting.loc[:,["assay_id", "PXD_accession", "pool_id"]]
#pool_formatting = pool_formatting[~(pool_formatting.pool_id == "no_annotation")]
pool_formatting.pool_id = pool_formatting.pool_id.astype(str)
pool_formatting.reset_index(drop = True, inplace = True)
pool_formatting

,assay_id,PXD_accession,pool_id
0,30960,PXD000533,1
1,30961,PXD000533,2
2,30962,PXD000533,1
3,30963,PXD000533,1
4,30964,PXD000533,2
...,...,...,...
1938,34107,PXD005940,70
1939,34108,PXD005940,69
1940,34109,PXD005940,69
1941,34110,PXD005940,69


Select files which need no pooling

In [118]:
no_pool = pool_formatting[pool_formatting.pool_id.isin(["no_annotation", "0", "/"])].assay_id.tolist()
len(no_pool)

431

Group the files that need pooling for each project

In [119]:
#list in list of pools
pools = pool_formatting[~pool_formatting.assay_id.isin(no_pool)].groupby(["PXD_accession", "pool_id"]).apply(lambda x: list(x.assay_id)).tolist()
len(pools)

108

Split data per assay or group of assays

In [120]:
DataFrameDict_no_pool = {elem: pd.DataFrame for elem in no_pool}
for key in DataFrameDict_no_pool.keys():
    DataFrameDict_no_pool[key] = protData[:][protData["assay_id"] == key]

In [121]:
DataFrameDict_pooled = {pool[0]: pd.DataFrame for pool in pools}
for pool in pools:
    DataFrameDict_pooled[pool[0]] = protData[:][protData.assay_id.isin(pool)]

In [122]:
print(f"No pool assays: {len(DataFrameDict_no_pool)}\nPooled assays: {len(DataFrameDict_pooled)}")
DataFrameDict_no_pool.update(DataFrameDict_pooled)
print(f"Concatened length: {len(DataFrameDict_no_pool)}")

No pool assays: 431
Pooled assays: 108
Concatened length: 539


In [123]:
DataFramaDict2 = DataFrameDict_no_pool.copy()

Calculate NSAF for each protein

In [124]:
for count, key in enumerate(DataFramaDict2.keys()):
    sumSaf = 0
    assay = DataFramaDict2[key]
    assay.pop("assay_id")

    #calculate sum of spectral counts for each protein
    grouped = DataFramaDict2[key].groupby("uniprot_id").sum().reset_index()
    seqAddedDF = pd.merge(grouped, seqData, on = "uniprot_id")
    seqAddedDF.insert(loc = 2, column = 'SAF', value = 0)
    seqAddedDF.insert(loc = 3, column = 'NSAF', value = 0)
    
    #Calculate SAF score for each protein by dividing sum of spectral counts by protein length
    for index, row in seqAddedDF.iterrows():
        saf = row['quantification']/row['length']
        seqAddedDF.loc[index, 'SAF'] = saf
        # calculate sum of SAF scores in assay
        sumSaf += saf

    # Calculate NSAF score by normalizing each SAF score
    seqAddedDF["NSAF"] = seqAddedDF["SAF"] / sumSaf
    
    del seqAddedDF['length']
    del seqAddedDF['quantification']
    del seqAddedDF['SAF']
    seqAddedDF.insert(loc = 0, column = 'assay_id', value = key)
    DataFramaDict2[key] = seqAddedDF

In [125]:
proteinData = pd.DataFrame()

for key in DataFramaDict2.keys():
    proteinData = proteinData.append(DataFramaDict2[key])

len(proteinData.assay_id.unique())

539

In [126]:
proteinData.head()

,assay_id,uniprot_id,NSAF
0,31056,A0AVT1,0.000333
1,31056,A0JNW5,0.000060
2,31056,A0MZ66,0.000278
3,31056,A2A3N6,0.000102
4,31056,A2RRP1,0.000111


In [128]:
#write NSAF proteome to file
proteinData.to_csv(path_or_buf = 'pooled_proteome_nsaf_1.csv', index = False)
df = pd.read_csv('pooled_proteome_nsaf_1.csv')
df.head()

,assay_id,uniprot_id,NSAF
0,31056,A0AVT1,0.000333
1,31056,A0JNW5,0.000060
2,31056,A0MZ66,0.000278
3,31056,A2A3N6,0.000102
4,31056,A2RRP1,0.000111


In [3]:
df = pd.read_csv("pooled_proteome_nsaf_1.csv")

assay_project_query = "SELECT assay_id, project_id FROM assay"
assay_project_data = pd.read_sql_query(assay_project_query, conn)

project_query = "SELECT project_id, PXD_accession FROM project"
project_data = pd.read_sql_query(project_query, conn)

assay_cell_query = "SELECT assay_id, cell_id FROM cell_to_assay"
assay_cell_data = pd.read_sql_query(assay_cell_query, conn)

cell_query = "SELECT cell_id, cell_line, tissue_type FROM cell"
cell_data = pd.read_sql_query(cell_query, conn)

full_df = pd.merge(df, assay_project_data, on = "assay_id")

full_df = pd.merge(full_df, assay_cell_data, on = "assay_id")
full_df = pd.merge(full_df, cell_data, on = "cell_id")
full_df = pd.merge(full_df, project_data, on = "project_id")
print(full_df.columns)
metadata = full_df[["assay_id", "cell_line", "PXD_accession", "tissue_type"]]

full_df

Index(['assay_id', 'uniprot_id', 'NSAF', 'project_id', 'cell_id', 'cell_line',
       'tissue_type', 'PXD_accession'],
      dtype='object')


,assay_id,uniprot_id,NSAF,project_id,cell_id,cell_line,tissue_type,PXD_accession
0,31056,A0AVT1,0.000333,1816,547,Jurkat,blood,PXD004280
1,31056,A0JNW5,0.000060,1816,547,Jurkat,blood,PXD004280
2,31056,A0MZ66,0.000278,1816,547,Jurkat,blood,PXD004280
3,31056,A2A3N6,0.000102,1816,547,Jurkat,blood,PXD004280
4,31056,A2RRP1,0.000111,1816,547,Jurkat,blood,PXD004280
...,...,...,...,...,...,...,...,...
1241200,31678,Q9Y6W3,0.000017,1834,577,SHSY5Y,brain,PXD010538
1241201,31678,Q9Y6W5,0.000164,1834,577,SHSY5Y,brain,PXD010538
1241202,31678,Q9Y6X9,0.000013,1834,577,SHSY5Y,brain,PXD010538
1241203,31678,Q9Y6Y0,0.000064,1834,577,SHSY5Y,brain,PXD010538


In [4]:
metadata = metadata.drop_duplicates().sort_values(["assay_id"]).reset_index(drop=True)

In [6]:
metadata.to_csv("metadata_NSAF_atlas.csv", index = False)

In [146]:
assay_atlas = pd.pivot_table(full_df, values = 'NSAF', index = 'assay_id', columns = 'uniprot_id').fillna(0).reset_index()
atlas = pd.merge(assay_atlas, metadata, on = 'assay_id')
atlas

,assay_id,A0A024RBG1,A0A075B6X5,A0A096LP01,A0A096LP49,A0A0A6YYK6,A0A0B4J1V0,A0A0B4J2F0,A0A0J9YWL9,A0A0J9YX94,...,Q9Y6X9,Q9Y6Y0,Q9Y6Y1,Q9Y6Y8,Q9Y6Y9,Q9Y6Z7,W5XKT8,cell_line,PXD_accession,tissue_type
0,30960,0.000010,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.000040,0.000062,0.000000,0.000088,0.000000,0.0,0.0,Hep3B,PXD000533,liver
1,30961,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.000042,0.000045,0.000001,0.000075,0.000000,0.0,0.0,Hep3B,PXD000533,liver
2,31008,0.000011,0.0,0.000061,0.0,0.0,0.000016,0.000000,0.000002,0.0,...,0.000024,0.000045,0.000000,0.000092,0.000024,0.0,0.0,MHCC97,PXD000533,liver
3,31009,0.000011,0.0,0.000080,0.0,0.0,0.000016,0.000000,0.000002,0.0,...,0.000035,0.000033,0.000000,0.000076,0.000024,0.0,0.0,MHCC97,PXD000533,liver
4,31056,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000350,0.000000,0.0,0.0,Jurkat,PXD004280,blood
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,33983,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.000161,0.000143,0.000000,0.000441,0.000000,0.0,0.0,SKOV3,PXD005940,ovary
535,33986,0.000189,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.000124,0.000107,0.000000,0.000291,0.000000,0.0,0.0,M14,PXD005940,skin
536,33989,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.000279,0.000024,0.000000,0.000389,0.000000,0.0,0.0,MCF7,PXD005940,breast
537,34076,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000169,0.000000,0.0,...,0.000018,0.000000,0.000000,0.000027,0.000000,0.0,0.0,CCRF-CEM,PXD005940,T-cell


In [147]:
atlas.to_csv('pooled_tsne_atlas1.csv', index=False)